In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from astropy import constants as const
from astropy import units as u

from scipy.integrate import quad

### Particle in an Infinite Well

Consider a particle of mass $m$ confined to a one-dimensional box of length $L$, where the potential $V(x)$ is given by:

$$
V(x) = 
\begin{cases} 
0 & \text{if } 0 < x < L \\
\infty & \text{otherwise} 
\end{cases} 
$$

#### 1. Setting up the Schrödinger Equation

For the regions where $V(x) = 0$, the time-independent Schrödinger equation is:

$$
-\frac{\hbar^2}{2m} \frac{d^2\psi(x)}{dx^2} = E \psi(x)
$$

#### 2. General Solution 

This is a second-order linear homogeneous differential equation with constant coefficients. The general solution is:

$$
\psi(x) = A \sin(kx) + B \cos(kx)
$$

Where $k^2 = \frac{2mE}{\hbar^2}$.

#### 3. Boundary Conditions

Given that the potential is infinite at $x = 0$ and $x = L$, the wavefunction must vanish at these points:

1) $\psi(0) = 0$ gives $B = 0$, so $\psi(x) = A \sin(kx)$.

2) $\psi(L) = 0$ gives:

$$
A \sin(kL) = 0
$$

For non-trivial solutions (i.e., $A \neq 0$), $\sin(kL)$ must be zero. Thus, $kL = n\pi$ where $n$ is a positive integer.

#### 4. Quantization of Energy

Using the relation $k^2 = \frac{2mE}{\hbar^2}$ and the condition $kL = n\pi$, we find:

$$
E_n = \frac{n^2 \pi^2 \hbar^2}{2mL^2}
$$

Where $E_n$ represents the energy of the $n$-th quantum state.

#### 5. Normalization

To find the normalization constant $A$, we use:

$$
\int_0^L |\psi(x)|^2 dx = 1
$$

For our wavefunction, this becomes:

$$
\int_0^L (A \sin(kx))^2 dx = 1
$$

Solving, we get:

$$
A = \sqrt{\frac{2}{L}}
$$

#### 6. Final Wavefunction

With the determined values of $A$ and $k$, our wavefunctions for the particle in an infinite well are:

$$
\psi_n(x) = \sqrt{\frac{2}{L}} \sin\left(\frac{n\pi x}{L}\right)
$$

Where $n = 1, 2, 3, ...$ 


In [ ]:
# energy for infinite well.
def energy (n, m, L):
    return (const.hbar**2/(2*m)*(n*np.pi/(L))**2).to(u.eV)

# time independent solution for infinite well.
def psi (x, n, L):
    return np.sqrt(2/L.value) * np.sin((n*np.pi*x/L).value)
                                      
# check normalization
def check_normalization(func, x_range, *args, **kwargs):
    integrand = lambda x_val: np.abs(func(x_val, *args, **kwargs))**2
    result, _ = quad(integrand, x_range[0], x_range[1])
    return result

In [ ]:
# choose L to be 1 nanometer (arbitrarily)
L = 1*u.nm

# create the space to be the exact width of the well.
x = np.linspace(0, L, 500)

# choose the mass to be 1eV at n=1 to show the n**2 relationship of energy levels.
m = 3.4254 * pow(10, -31) *u.kg #((const.hbar*np.pi/L)**2/(2*u.eV)).to(u.kg)

In [ ]:
# First plot the time independent solutions at their energy levels
n_max = 5
for n in range(1, n_max):
    E = energy(n, m, L).value
    psi_n = psi(x, n, L)
    label_text = f"n={n}, E={E:.2f} eV"  # Create a label for each line
    plt.plot(x, psi_n+E, label=label_text)
    norm_check = check_normalization(psi, (0, L.value), n, L)
    print(f"Normalization result for psi_{n}: {norm_check:.2f}")
    
# Representing the infinite well:
# Drawing vertical lines at z=0 and z=L
plt.axvline(0, color='black', lw=2)
plt.axvline(L.value, color='black', lw=2)

# Optional: setting y-limits for a better view
plt.ylim(-energy(1, m, L).value, energy(n_max-1, m, L).value * 1.5)

plt.xlabel('Position (nm)')
plt.ylabel('Energy (eV) / Wavefunction')
plt.title('Particle in an Infinite Well')
plt.legend()
plt.grid(True)
plt.show()


### Time-Dependent Solution for the Infinite Well

Given the time-independent solutions for the particle in an infinite well, we can introduce the time-dependence through the time-evolution operator in quantum mechanics. The general form of a time-dependent solution is:

$$
\Psi(x, t) = \psi(x) e^{-i \frac{E}{\hbar} t}
$$

Where:
- $ \Psi(x, t) $ is the time-dependent wave function.
- $ \psi(x) $ is the time-independent solution we derived earlier.
- $ E $ is the energy of the state in consideration.

#### For the Infinite Well:

We already have:

$$
\psi_n(x) = \sqrt{\frac{2}{L}} \sin\left(\frac{n\pi x}{L}\right)
$$

And:

$$
E_n = \frac{n^2 \pi^2 \hbar^2}{2mL^2}
$$

Substituting these into our general time-dependent equation, we get:

$$
\Psi_n(x, t) = \sqrt{\frac{2}{L}} \sin\left(\frac{n\pi x}{L}\right) \exp\left(-i \frac{E_n t}{\hbar}\right)
$$

This equation provides a complete description of the particle in the well as a function of position and time. As $ t $ varies, the phase of the wave function changes, but the probability density $ |\Psi(x, t)|^2 $ remains constant in time.


In [ ]:
# time independent solution for infinite well.
def psi_time_dep(x, n, m, L, t):
    # Time-independent part
    psi_n = psi (x, n, L)
    
    # Energy of the nth state
    E_n = energy(n, m, L)  
    
    return psi_n * np.exp(-1j*E_n/const.hbar*t)
 

In [ ]:
# Examine how the real and imaginary parts of the wave function vary over time
# Overlay the probability density, but note this remains unchanged with time.
fig, ax = plt.subplots(figsize=(10, 8))

n = 2 # Try out different values of n.
t_0 = 0 *u.s

# Plot the probability density, which remains unchanged with time
wavefunc = psi_time_dep(x, n, m, L, t_0)
prob_density = wavefunc * np.conj(wavefunc)

# Take only the real component - discards small, insignificant imaginary parts due to numerical errors.
prob_density = prob_density.real  
ax.fill_between(x.value, prob_density, color='gray', alpha=0.3, label='Probability Density')

line_real, = ax.plot(x, psi_time_dep(x, n, m, L, t_0).real, label='Real Part')
line_imag, = ax.plot(x, psi_time_dep(x, n, m, L, t_0).imag, '--', label='Imaginary Part')
norm_check = check_normalization(psi_time_dep, (0, L.value), n, m, L, t_0)
print(f"Normalization result for psi_time_dep_{n}: {norm_check:.2g}")

ax.set_xlim(0, L.value)
ax.set_ylim(-1.5, 3.0)
ax.set_title(f'Time Evolution of Wave Function for n={n}')
ax.set_xlabel('Position (nm)')
ax.set_ylabel('Amplitude')
ax.legend(loc='upper right', facecolor='white', edgecolor='black')
ax.grid(True)

def update(t):
    wavefunc = psi_time_dep(x, n, m, L, t*u.s)
    line_real.set_ydata(wavefunc.real)
    line_imag.set_ydata(wavefunc.imag)
    return line_real, line_imag

E = energy(n, m, L)
T = (2 * np.pi * const.hbar / E).to(u.s).value  # Set time range for full period.
ani = FuncAnimation(fig, update, frames=np.linspace(0, T, 100), blit=True, repeat=True)
plt.close(fig)

# Display the animation in this cell only as an HTML5 video
HTML(ani.to_jshtml())

In [ ]:
def super_position(x, n1, n2, m, L, t):
    # Note need to normalize given we have a super position
    def integrand(x_val):
        sum_wavefunctions = psi_time_dep(x_val, n=n1, m=m, L=L, t=t) + psi_time_dep(x_val, n=n2, m=m, L=L, t=t)
        return np.abs(sum_wavefunctions)**2

    # Integrate using quad
    integral, _ = quad(integrand, 0, L.value)

    norm_const = 1/np.sqrt(integral)
    
    return norm_const*(psi_time_dep(x, n1, m, L, t) + psi_time_dep(x, n2, m, L, t))
    
def prob_density_super_position(x, n1, n2, m, L, t):
    psi_super = super_position(x, n1, n2, m, L, t)
    
    # Take only the real component - discards small, insignificant imaginary parts due to numerical errors.
    prob_density = (psi_super * np.conj(psi_super)).real
    
    return prob_density

n1=1
n2=2

fig, ax = plt.subplots(figsize=(10, 8))
line_prob_density, = ax.plot(x, prob_density_super_position(x, n1, n2, m, L, t_0).real, label='Prob Density')

result_time = check_normalization(super_position, (0, L.value), n1, n2, m, L, t_0)
print(f"Normalization result super position of n1={n1} and n2={n2}: {result_time:.2g}")

ax.set_xlim(0, L.value)
ax.set_ylim(-1.0, 5.0)
ax.set_title(f'Time Evolution of Wave Function super position of n1={n1} and n2={n2}')
ax.set_xlabel('Position (nm)')
ax.set_ylabel('Amplitude')
ax.legend(loc='upper right', facecolor='white', edgecolor='black')
ax.grid(True)

def update(t):
    prob_density = prob_density_super_position(x, n1, n2, m, L, t*u.s)
    line_prob_density.set_ydata(prob_density)
    return line_prob_density,

#  time period required for a full cycle depends on the difference in the energies of the two states in superposition.
E = energy(n2, m, L) - energy(n1, m, L)
T = (2 * np.pi * const.hbar / E).to(u.s).value  # Set time range for full period.
ani = FuncAnimation(fig, update, frames=np.linspace(0, T, 100), blit=True, repeat=True)
plt.close(fig)

# Display the animation in this cell only as an HTML5 video
HTML(ani.to_jshtml())